In [66]:
#Use this if you don't have all the packages installed, this will add them.
import Pkg
Pkg.add(["IterTools","LinearAlgebra", "Random", "SparseArrays", "COSMO", "JuMP"])

   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [37e2e46d] + LinearAlgebra
  [2f01184e] + SparseArrays
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [1]:
using IterTools
using LinearAlgebra, Random, SparseArrays, COSMO, JuMP

In [23]:
#Get all Pauli strings of weight W
function enumerate_paulistrings(one_paulis::Vector{Char}, n::Int, W::Int)::Vector{String}
    res = String[]
    #Get all lists of W distinct indices
    for indices = IterTools.subsets(1:n, W)
        pauli_char_array = repeat(['I'],n)
        #And all 3^W Pauli strings on those indices
        for pauli_type = Iterators.product(repeat([one_paulis], W)...)
            for (i,ci) = enumerate(indices)
                pauli_char_array[ci] = pauli_type[i]
            end
            push!(res, String(pauli_char_array))
        end
    end
    return res
end

@assert length(enumerate_paulistrings(['F','G','H','J','K'],20,3)) == (5^3) * (20*19*18)/(6)
@assert length(enumerate_paulistrings(['X','Y'],5,6)) == 0 #nothing of weight higher than system size

#Multiplies two strings, returning a new string and a phase.
#Currently only works for qubits with normal XYZ labels.
function multiply_paulistrings(strA::String, strB::String)::Tuple{String, Complex}
    @assert length(strA) == length(strB) "Paulis must be same length"
    res_char_array = repeat(['I'],length(strA))
    acc_phase = Complex(1) #1 + 0im
    for (i,(a,b)) = enumerate(zip(strA, strB))
        if a==b
            continue #I^2=X^2=Y^2=Z^2 = I
        elseif a=='I'
            res_char_array[i] = b
        elseif b=='I'
            res_char_array[i] = a
        elseif a=='X' && b=='Y'
            res_char_array[i] = 'Z'
            acc_phase *= 1im
        elseif a=='Y' && b=='Z'
            res_char_array[i] = 'X'
            acc_phase *= 1im
        elseif a=='Z' && b=='X'
            res_char_array[i] = 'Y'
            acc_phase *= 1im
        elseif a=='X' && b=='Z'
            res_char_array[i] = 'Y'
            acc_phase *= -1im
        elseif a=='Y' && b=='X'
            res_char_array[i] = 'Z'
            acc_phase *= -1im
        elseif a=='Z' && b=='Y'
            res_char_array[i] = 'X'
            acc_phase *= -1im
        else
            error("Bad characters "+a+" and "+b)
        end
    end
    return (String(res_char_array), acc_phase)
end

@assert multiply_paulistrings("IIX","YIX") == ("YII",1)
@assert multiply_paulistrings("IXXIII","YYIIII") == ("YZXIII",1im)

function make_qubit_SDP_vars(q, n, d, k)
    @assert k <= 2*d "Observable order k must be at most 2 times pseudostate level d"
    @assert 2*d <= n+1 "Level d is higher than make sense for only n qudits; only need ceil(n/2)."

    #Our nontrivial one-local operators (for qubits, the three Paulis). Should be tomographically complete
    one_paulis = ['X','Y','Z']
    @assert length(one_paulis) == q^2 - 1 "Incorrect number of local operators"

    #List of lists of Pauli strings for the 'rows' and 'columns' of our pseudostate. The Wth element is
    # all the weight-W Pauli strings we need. Goes up to weight d.
    row_ops = Vector{String}[]
    #And one big list of all operators in the whole matrix, up to weight 2d.
    mat_ops = String[]

    #Get the operators for each weight. Also, the trivial "III" column.
    row_III = repeat("I",n)
    for W = 1:d
        push!(row_ops, enumerate_paulistrings(one_paulis, n, W))
    end
    row_ops_flat = collect(Iterators.flatten(row_ops))

    mat_ops = copy(row_ops_flat)
    for W = d+1:2d
        append!(mat_ops, enumerate_paulistrings(one_paulis, n, W))
    end
    return (row_ops, row_ops_flat, mat_ops, row_III)
end

function make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)
    #Build the matrix of these variables for the pseudostate.
    #First allocate the variables, all real expectations in the range -1 to 1:
    model = JuMP.Model(optimizer_with_attributes(COSMO.Optimizer, "verbose" => false));

    @variable(model, -1 ≤ vars[mat_ops] ≤ 1)
    #Now vars["XII"] is the XII expectation value.

    #Next place them in a matrix, with complex coefficients. With the complex coefficients, these are
    #type GenericAffExpr{ComplexF64, VariableRef}.
    pseudo_len = length(row_ops_flat)
    pseudo_mat = zeros(GenericAffExpr{ComplexF64, VariableRef}, 1+pseudo_len, 1+pseudo_len)
    for i=0:pseudo_len, j=0:pseudo_len
        if i == 0
            if j == 0
                pseudo_mat[1+i, 1+j] += 1
            else
                jstr = row_ops_flat[j]
                jvar = vars[jstr]
                pseudo_mat[1+i, 1+j] += jvar
            end
        elseif j == 0
            istr = row_ops_flat[i]
            ivar = vars[istr]
            pseudo_mat[1+i, 1+j] += ivar
        else
            istr = row_ops_flat[i]
            jstr = row_ops_flat[j]
            ijstr, phase = multiply_paulistrings(istr, jstr)
            if ijstr == row_III
                ijvar = 1
            else
                ijvar = vars[ijstr]
            end
            pseudo_mat[1+i, 1+j] += phase * ijvar
        end
    end

    #Need it as a LinearAlgebra.Hermitian type for JuMP.
    pseudo_mat_herm = Hermitian(pseudo_mat)
    @assert pseudo_mat_herm == pseudo_mat "Pseudostate SDP wasn't Hermitian"

    #Add constraint to the model.
    @constraint(model, pseudo_mat_herm in HermitianPSDCone())
    
    return vars, model
end

function pseudostate_bound_expr(obj_expr, model = (0.0+1*obj_expr).terms.keys[1].model)
    #Get measurement to predict (lower bound):
    @objective(model, Min, obj_expr)
    JuMP.optimize!(model)
    if JuMP.primal_status(model) != FEASIBLE_POINT
        error("Infeasible input data, status = "*string(JuMP.termination_status(model)))
    end
    min_val = JuMP.value(obj_expr)
    println("Min value = ",min_val)

    @objective(model, Max, obj_expr)
    JuMP.optimize!(model)
    if JuMP.primal_status(model) != FEASIBLE_POINT
        error("Infeasible input data, status = "*string(JuMP.termination_status(model)))
    end
    max_val = JuMP.value(obj_expr)

    result_interval = [min_val, max_val]
    return result_interval
end

pseudostate_bound_expr (generic function with 2 methods)

### 3 Qubits

In [20]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 3 #3 qubits
d = 1 #level-1 pseudostate
k = 2 #looking at observables of at most order 2
;

In [18]:
(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k);

vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

#PAC framework:
#Add in measurement data as constraints:
@constraint(model, 0.3 ≤ vars["IIX"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIY"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["XIY"] ≤ -0.38)
@constraint(model, 0.3 ≤ vars["YIZ"] ≤ 0.4)
@constraint(model, 0.2 ≤ vars["ZIX"] ≤ 0.25)
@constraint(model, 0.5 ≤ vars["IYZ"] ≤ 0.55)
@constraint(model, 0.72 ≤ vars["XZI"] ≤ 0.74)
@constraint(model, 0.72 ≤ vars["YXI"] ≤ 0.74)

@time res_interval = pseudostate_bound_expr(vars["IIX"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = 0.30000000000000004
  0.053259 seconds (19.83 k allocations: 7.695 MiB)
Possible interval: [0.30000000000000004, 0.35480327522055477]
Time for second solve was 0.03172111511230469


Rerunning the same data at pseudostate level 2 now:

In [19]:
d = 2
(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k)
vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

#PAC framework:
#Add in measurement data as constraints:
@constraint(model, 0.3 ≤ vars["IIX"] ≤ 0.6)
@constraint(model, 0.79 ≤ vars["IIY"] ≤ 0.81)
@constraint(model, -0.4 ≤ vars["XIY"] ≤ -0.38)
@constraint(model, 0.3 ≤ vars["YIZ"] ≤ 0.4)
@constraint(model, 0.2 ≤ vars["ZIX"] ≤ 0.25)
@constraint(model, 0.5 ≤ vars["IYZ"] ≤ 0.55)
@constraint(model, 0.72 ≤ vars["XZI"] ≤ 0.74)
@constraint(model, 0.72 ≤ vars["YXI"] ≤ 0.74)

@time res_interval = pseudostate_bound_expr(vars["IIX"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = 0.30000000000000004
  1.768557 seconds (61.64 k allocations: 88.175 MiB, 0.93% gc time)
Possible interval: [0.30000000000000004, 0.3034448202654543]
Time for second solve was 0.4734208583831787


### 8 qubits

In [54]:
function test_11_qubits(model, vars)
    #PAC framework:
    #Add in measurement data as constraints:
    @constraint(model, -0.9 ≤ vars["IIXIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIYIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["XIIIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["YIIIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["IIZIIIIX"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIIIXIIZ"] ≤ 0.81)
    @constraint(model, -0.4 ≤ vars["IIYIIIXI"] ≤ -0.38)
    @constraint(model, 0.3 ≤ vars["IYIZIIII"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IZIXIIII"] ≤ 0.25)
    @constraint(model, 0.3 ≤ vars["IIIIIYIZ"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IIIIIZIX"] ≤ 0.25)
    @constraint(model, 0.5 ≤ vars["IIIYZIII"] ≤ 0.55)
    @constraint(model, 0.5 ≤ vars["IIIIIYZI"] ≤ 0.55)
    @constraint(model, 0.72 ≤ vars["IIIIIXZI"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["YXIIIIII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIXZII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIYXII"] ≤ 0.74)
end

test_11_qubits (generic function with 1 method)

In [55]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 8 #8 qubits
d = 1 #level-1 pseudostate
k = 2 #looking at observables of at most order 2

(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k);


vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

test_11_qubits(model, vars)

@time res_interval = pseudostate_bound_expr(vars["IIXIIIII"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = -0.5346960295726031
  0.248197 seconds (63.75 k allocations: 38.414 MiB)
Possible interval: [-0.5346960295726031, 0.5346923598918945]
Time for second solve was 0.11071610450744629


In [63]:
function test_11_qubits_2(model, vars)
    #PAC framework:
    #Add in measurement data as constraints:
    @constraint(model, -0.9 ≤ vars["IIXIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["IIYIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["XIIIIIII"] ≤ 0.6)
    @constraint(model, 0.79 ≤ vars["YIIIIIII"] ≤ 0.81)
    @constraint(model, 0.3 ≤ vars["IIZIIIIX"] ≤ 0.6)
#     @constraint(model, 0.79 ≤ vars["IIIIXIIZ"] ≤ 0.81)
    @constraint(model, -0.4 ≤ vars["IIYIIIXI"] ≤ -0.38)
#     @constraint(model, 0.3 ≤ vars["IYIZIIII"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IZIXIIII"] ≤ 0.25)
#     @constraint(model, 0.3 ≤ vars["IIIIIYIZ"] ≤ 0.4)
    @constraint(model, 0.2 ≤ vars["IIIIIZIX"] ≤ 0.25)
#     @constraint(model, 0.5 ≤ vars["IIIYZIII"] ≤ 0.55)
    @constraint(model, 0.5 ≤ vars["IIIIIYZI"] ≤ 0.55)
#     @constraint(model, 0.72 ≤ vars["IIIIIXZI"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["YXIIIIII"] ≤ 0.74)
#     @constraint(model, 0.72 ≤ vars["IIIIXZII"] ≤ 0.74)
    @constraint(model, 0.72 ≤ vars["IIIIYXII"] ≤ 0.74)
end

test_11_qubits_2 (generic function with 1 method)

In [64]:
q = 2 #local dimension 2 (qu-k-its == qubits)
n = 8 #8 qubits
d = 2 #level-2 pseudostate
k = 2 #looking at observables of at most order 2

(row_ops, row_ops_flat, mat_ops, row_III) = make_qubit_SDP_vars(q, n, d, k);


vars, model = make_pseudostate_model(row_ops, row_ops_flat, mat_ops, row_III)

test_11_qubits_2(model, vars)

@time res_interval = pseudostate_bound_expr(vars["IIXIIIII"])
println("Possible interval: ",res_interval)
println("Time for second solve was ",solve_time(model))

Min value = -0.5347273453036312
 68.357837 seconds (2.79 M allocations: 3.120 GiB, 0.13% gc time)
Possible interval: [-0.5347273453036312, 0.5347282391754715]
Time for second solve was 37.02287316322327
